In [1]:
# coding: utf-8

In [2]:
get_ipython().system(u'pip install nilearn')

  Obtaining dependency information for nilearn from https://files.pythonhosted.org/packages/d3/6d/f9e6f3e38bf6d8d2986c6a426f69f3b45191367383494a515f45dd748692/nilearn-0.10.2-py3-none-any.whl.metadata
  Obtaining dependency information for lxml from https://files.pythonhosted.org/packages/50/ba/cb7bc9728a3be4e00dfd658fc76dc64fd9dbc3d5492ff44cda70574329c6/lxml-4.9.3-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.3 MB 435.7 kB/s eta 0:00:08
      --------------------------------------- 0.1/3.3 MB 491.5 kB/s eta 0:00:07
     - -------------------------------------- 0.1/3.3 MB 476.3 kB/s eta 0:00:07
     - -------------------------------------- 0.1/3.3 MB 552.2 kB/s eta 0:00:06
     - -------------------------------------- 0.1/3.3 MB 532.5 kB/s eta 0:00:06
     -- ------------------------------------- 0.2/3.3 MB


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import numpy as np
from scipy import linalg

from nilearn import datasets
from nilearn.input_data import NiftiMasker

from nilearn.image import smooth_img
import numpy as np
import cv2

import keras

n_subjects = 416

oasis_dataset = datasets.fetch_oasis_vbm(n_subjects=n_subjects)
gray_matter_map_filenames = oasis_dataset.gray_matter_maps
gm_imgs = gray_matter_map_filenames

c:\Users\vasan\Downloads\Programs\Python\Python310\lib\site-packages\nilearn\input_data\__init__.py:23: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)
c:\Users\vasan\Downloads\Programs\Python\Python310\lib\site-packages\nilearn\datasets\struct.py:688: UserWarning: Only 403 subjects are available in the DARTEL-normalized version of the dataset. All of them will be used instead of the wanted 416
  warnings.warn(
c:\Users\vasan\Downloads\Programs\Python\Python310\lib\site-packages\nilearn\datasets\struct.py:852: UserWarning: `legacy_format` will default to `False` in release 0.11. Dataset fetchers will then return pandas dataframes by default instead of recarrays.
  warnings.warn(_LEGACY_FORMAT_MSG)


In [ ]:
cdr = oasis_dataset.ext_vars['cdr'].astype(float)
cdr_numpy_arr = np.array(cdr)
for i in range(len(cdr_numpy_arr)):
    if(np.isnan(cdr_numpy_arr[i])): 
        cdr_numpy_arr[i] = 1
    elif(cdr_numpy_arr[i] > 0.0):
        cdr_numpy_arr[i] = 1

In [11]:
imgArr = []

for imgUrl in gray_matter_map_filenames:
    result_img = smooth_img(imgUrl, fwhm=1)
    imgArr.append(result_img.get_fdata())

In [12]:
x_train = []
x_test = []

y_train = []
y_test = []

rshapedImgArr = []

for img in imgArr:
    newImg = [cv2.resize(each_slice,(50,50)) for each_slice in img]#Reducing slice count
    newImg = np.array(newImg)
    rshapedImgArr.append(newImg)
    
label = cdr_numpy_arr

In [13]:
label = keras.utils.to_categorical(cdr_numpy_arr, 2)

much_data = []

for num, img in enumerate(rshapedImgArr):
    much_data.append([img,label[num]])

In [15]:
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

IMG_SIZE_PX_X = 50
IMG_SIZE_PX_Y = 50
SLICE_COUNT = 91

n_classes = 2
batch_size = 10

x = tf.placeholder('float')
y = tf.placeholder('float')

keep_rate = 0.8

In [16]:
def conv3d(x, W):
    conv = tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')
    conv = tf.nn.dropout(conv, 0.5)
    return conv

def maxpool3d(x):
    #                        size of window         movement of window as you slide about
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')

In [17]:
def convolutional_neural_network(x):
    #                # 5 x 5 x 5 patches, 1 channel, 32 features to compute.
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,1,32])),
                #       5 x 5 x 5 patches, 32 channels, 64 features to compute.
                'W_conv2':tf.Variable(tf.random_normal([3,3,3,32,64])),
                #                                  64 features
                'W_fc':tf.Variable(tf.random_normal([248768,1024])),
                'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
                'b_conv2':tf.Variable(tf.random_normal([64])),
                'b_fc':tf.Variable(tf.random_normal([1024])),
                'out':tf.Variable(tf.random_normal([n_classes]))}

    #                            image X      image Y        image Z
    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX_X, IMG_SIZE_PX_Y, SLICE_COUNT, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)


    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)

    fc = tf.reshape(conv2,[-1, 248768])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output

In [ ]:
# train_data = much_data[:-333]
# validation_data = much_data[-83:]

from sklearn.model_selection import train_test_split

def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost)
    
    file = open("Output.txt", "w")
    
    # hm_epochs = 1000
    hm_epochs = 50

    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        successful_runs = 0
        total_runs = 0
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            
            train_data, validation_data = train_test_split(much_data, train_size=0.8)
            
            for data in train_data:
                total_runs += 1
                try:
                    X = data[0]
                    Y = data[1]
                    _, c = sess.run([optimizer, cost], feed_dict={x: X, y: Y})
                    epoch_loss += c
                    successful_runs += 1
                except Exception as e:
                    pass
                    #print(str(e))
            
            print(f"Epoch {epoch+1} completed out of {hm_epochs} loss : {epoch_loss}")
            file.write(f"Epoch {epoch+1} completed out of {hm_epochs} loss : {epoch_loss}")

            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

            print(f"Accuracy : {accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]})}")
            file.write(f"Accuracy : {accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]})}")
            
            save_path = saver.save(sess, "model.ckpt")
            print(f"Model saved in file: {save_path}")

        print('Done. Finishing accuracy : ')
        print(f"Accuracy : {accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]})}")
        
        print(f"fitment percent : {successful_runs/total_runs}")
        
        file.write(f"Done. Finishing accuracy : ")
        file.write(f"Accuracy : {accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]})}")
        
        file.write(f"fitment percent : {successful_runs/total_runs}")

        save_path = saver.save(sess, "model.ckpt")
        print(f"Final model saved in file : {save_path}")

train_neural_network(x)